In [164]:
!pip install haversine

  Running setup.py bdist_wheel for haversine ... done
  Stored in directory: /Users/rajchakrabarty/Library/Caches/pip/wheels/be/a3/4e/2e073dc725bf9b430e2d42931c55e5be2be4cdc6e325e17677
Successfully built haversine
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import keys
import pandas as pd
import requests
from pprint import pprint
import json
import numpy as np

In [3]:
screens = pd.read_csv('screen geocoded.csv')
screens.drop('Unnamed: 0', axis=1, inplace=True)
screens.shape

(628, 12)

In [4]:
screens.head()

,Screen Design: Screen Design,Opportunity: Opportunity Name,Screen Placement Address,Screen Addresss (City),Screen Address (State),Screen Address (Zip),Install Stage,Region,Screen Type,lat,lng,Address
0,Azure Park (JPM & ARP),JPM & ARP-Azure on the Park ATL,1020 Piedmont Ave NE,Atlanta,GA,30309,Customer Live,ATL,Public Screen,33.782524,-84.380599,1020 Piedmont Ave NE Atlanta GA 30309
1,Banyan Street Capital - ATL - 191 Peachtree Tower,Banyan Street Capital - ATL - 191 Peachtree Tower,191 Peachtree St. NE,Atlanta,GA,30303,Customer Live,ATL,Public Screen,33.749514,-84.394477,191 Peachtree St. NE Atlanta GA 30303
2,athenahealth (WeDriveU) Atlanta,WeDriveU-AthenaHealth - Arsenal Campus- Revisit,"675 Ponce de Leon Ave NE, Suite 9000",Atlanta,GA,30308,Customer Live,ATL,Public Screen,33.772828,-84.365784,"675 Ponce de Leon Ave NE, Suite 9000 Atlanta G..."
3,Buckhead at Lenox Square,Parmenter - ATL - Buckhead Tower at Lenox Square,3399 Peachtree Road NE,Atlanta,GA,30326,Customer Live,ATL,Public Screen,33.846098,-84.359549,3399 Peachtree Road NE Atlanta GA 30326
4,Seven Oaks - ATL - 4004 Perimeter Summit,Seven Oaks - ATL - 4004 Perimeter Summit,4004 Perimeter Summit Pkwy,Atlanta,GA,30319,Customer Live,ATL,Public Screen,33.915213,-84.340141,4004 Perimeter Summit Pkwy Atlanta GA 30319


In [5]:
#look at number that are missing lats and longs
print (screens[screens.lat.isnull()==True].shape)
print (screens[screens.lat.isnull()==False].shape)
print ()

(51, 12)
(577, 12)



In [ ]:
########################################

In [7]:
#add close date from model-data file
model_data = pd.read_csv('model_data.csv')

In [8]:
model_data.head()

,id,name,close_date,stage,address,lat,lng,ts_response,mobility_score,carshare,bikeshare,ridehailing,masstransit,lat_lng,closest_ts,within_one_tenth,within_one_half,within_one,within_five
0,0061a00000GcQdL,525 Water St SW- CMC Mgmt,3/8/18,Contract Sent,525 Water St SW Washington United States Distr...,38.876059,-77.020240,{'data': {'mobilityScore': {'latitude': 38.876...,86.0,12.190237,21.710327,21.000000,25.347332,"(38.876059000000012, -77.020239899999979)",7.073231e-13,1.0,6.0,11.0,72.0
1,0061a00000Mi8dC,Bozzuto - NJ - Park + Garden,6/30/18,Invoice Sent,1450 Garden St Hoboken United States New Jerse...,40.754448,-74.028046,{'data': {'mobilityScore': {'latitude': 40.754...,85.0,3.435388,25.069638,15.000000,33.883246,"(40.754448199999999, -74.028046000000003)",1.927253e-01,0.0,1.0,3.0,31.0
2,0061a00000M4PAU,Greystar- The Cameron,11/17/17,Deal Signed,"8710 Cameron St, Silver Spring, MD 20910",38.999289,-77.028885,{'data': {'mobilityScore': {'latitude': 38.999...,91.0,2.921386,8.266361,12.115385,70.074879,"(38.9992886, -77.028885400000021)",2.301765e-01,0.0,4.0,6.0,11.0
3,0061a00000MgelJ,JLL- 2035 NW Front Ave,1/31/18,Proposal Sent,"1120 NW Couch St #500, Portland, OR 97209",45.537557,-122.688094,{'data': {'mobilityScore': {'latitude': 45.537...,100.0,0.000000,0.000000,21.000000,1.468259,"(45.537556700000003, -122.6880937)",6.848705e-01,0.0,0.0,1.0,6.0
4,0061a00000KOF8z,International Monetary Fund- Spring Conference,1/31/18,Closed Won,"1900 Pennsylvania Ave NW, Washington, DC 20431",38.898940,-77.044101,{'data': {'mobilityScore': {'latitude': 38.898...,100.0,14.075039,40.000111,21.000000,87.423132,"(38.898939599999999, -77.044101499999996)",2.050043e-01,0.0,4.0,11.0,92.0


In [9]:
close_dates = model_data[['name'
                          ,'close_date'
                          #,'address'
                         ]]

In [10]:
close_dates.shape

(1683, 2)

In [11]:
screens = screens.merge(close_dates, how='left'
                        ,right_on='name'
                        ,left_on='Opportunity: Opportunity Name'
                        #,indicator=True
                       )

In [12]:
#why did the number of records go from 628 to 649? are there dups?
print(screens.shape)
print(screens[screens.close_date.isnull()==False].shape)
print(screens[screens.close_date.isnull()==True].shape)

(649, 14)
(407, 14)
(242, 14)


In [14]:
screens.drop(['name'], axis=1, inplace=True)

In [13]:
screens[screens.close_date.isnull()==True].head(100)

,Screen Design: Screen Design,Opportunity: Opportunity Name,Screen Placement Address,Screen Addresss (City),Screen Address (State),Screen Address (Zip),Install Stage,Region,Screen Type,lat,lng,Address,name,close_date
2,athenahealth (WeDriveU) Atlanta,WeDriveU-AthenaHealth - Arsenal Campus- Revisit,"675 Ponce de Leon Ave NE, Suite 9000",Atlanta,GA,30308,Customer Live,ATL,Public Screen,33.772828,-84.365784,"675 Ponce de Leon Ave NE, Suite 9000 Atlanta G...",NaN,NaN
5,Engrain - Post Alexander - ATL - 600 Phipps Blvd,Post Alexander - ATL,600 Phipps Boulevard,Atlanta,GA,30326,Customer Live,ATL,Public Screen,33.853462,-84.365255,600 Phipps Boulevard Atlanta GA 30326,NaN,NaN
9,Local on 14th- Matrix Residential,Matrix Residential - ATL - Local on 14th,455 14th St. NW,Atlanta,GA,30318,Customer Live,ATL,Public Screen,33.786480,-84.402097,455 14th St. NW Atlanta GA 30318,NaN,NaN
11,Seven Oaks - ATL - 3003 Perimeter Summit,Seven Oaks - ATL - 3003 Perimeter Summit,3003 Perimeter Summit,Atlanta,GA,30319,Customer Live,ATL,Public Screen,33.913781,-84.342588,3003 Perimeter Summit Atlanta GA 30319,NaN,NaN
12,Seven Oaks - ATL - 1001 Perimeter Summit,Seven Oaks - ATL - 1001 Perimeter Summit,1001 Perimeter Summit,Atlanta,GA,30319,Customer Live,ATL,Public Screen,33.915159,-84.340140,1001 Perimeter Summit Atlanta GA 30319,NaN,NaN
16,Seven Oaks - ATL - 2002 Perimeter Summit,Seven Oaks - ATL - 2002 Perimeter Summit,2002 Perimeter Summit,Atlanta,GA,30319,Customer Live,ATL,Public Screen,33.914542,-84.341901,2002 Perimeter Summit Atlanta GA 30319,NaN,NaN
25,Halstead Malden Square Apartments,Priderock Capital - Halstead Malden Square Apa...,480 Main St,Malden,MA,02148,Customer Live,BOS,Public Screen,42.427693,-71.067326,480 Main St Malden MA 02148,NaN,NaN
26,Watermark Seaport - 85 Seaport Blvd,Greystar - Watermark 85 Seaport,85 Seaport Blvd,Boston,MA,NaN,Customer Live,BOS,Public Screen,NaN,NaN,NaN,NaN,NaN
28,MIT School of Engineering,MIT Engineering Building,77 Mass Ave,Cambridge,MA,02139,Customer Live,BOS,Public Screen,42.359244,-71.093139,77 Mass Ave Cambridge MA 02139,NaN,NaN
29,Cresset Group - 65 Grove Street - Watertown,Cresset Group - 65 Grove Street - Watertown,65 Grove Street,Watertown,MA,02472,Customer Live,BOS,Public Screen,42.370026,-71.153856,65 Grove Street Watertown MA 02472,NaN,NaN


unmatching screen addresses:
WeDriveU-AthenaHealth - Arsenal Campus- Revisit

Post Alexander - ATL

Matrix Residential - ATL - Local on 14th

Seven Oaks - ATL - 3003 Perimeter Summit



In [135]:
model_data[model_data.lat==33.772828]

,id,name,close_date,stage,address,lat,lng,ts_response,mobility_score,carshare,bikeshare,ridehailing,masstransit


It looks like this screen didn't exist in close dates

In [15]:
model_data[model_data.stage=='Closed Won'].shape

(295, 19)

In [156]:
screens.drop_duplicates(['lat'], keep='first', inplace=True)

In [154]:
screens.to_csv('screens_and_close_dates.csv', index=False)

In [ ]:
################################################

In [173]:
from haversine import haversine

In [256]:
def find_closest(point, list_of_points, debug=False):
    distances = []
    for i in list_of_points:
        #if it is a different point, calculate the distance
        if point != i:
            dist = haversine(point, i)
            if debug:
                print (point, i, dist)
            distances.append(dist)
    #return the smallest distance 
    #do we want to set a max here?
    return (distances[np.argmin(distances)])
    

In [258]:
point = (38.876059, -77.020240)
list_of_points = [(40.754448,-74.028046), (38.999289,-77.028885), (38.876059, -77.020240)]
find_closest(point, list_of_points,True)

(38.876059, -77.02024) (40.754448, -74.028046) 330.0218885526628
(38.876059, -77.02024) (38.999289, -77.028885) 13.722935990827294


13.722935990827294

In [215]:
point = (38.876059, -77.020240)
list_of_points = [(40.754448,-74.028046), (38.999289,-77.028885)]
find_closest(point, list_of_points)

(38.876059, -77.02024) (40.754448, -74.028046) 330.0218885526628
(38.876059, -77.02024) (38.999289, -77.028885) 13.722935990827294


13.722935990827294

In [327]:
def find_within_distance(point, list_of_points, radius=.5):
    count = 0
    for i in list_of_points:
        dist = haversine(point, i)
        #if point == i:
        #    print('found self', i)
        if point != i:
            if dist <= radius:
                count +=1
    return (count)

In [251]:
point = (38.876059, -77.020240)
list_of_points = [(40.754448,-74.028046), (38.999289,-77.028885)]
find_within_distance(point, list_of_points,14)

1

In [210]:
model_data.shape

(1683, 13)

In [214]:
print(screens.shape)
print(screens[screens.close_date.isnull()==False].shape)
print(screens[screens.close_date.isnull()==True].shape)

(544, 13)
(335, 13)
(209, 13)


In [232]:
#build a list of all points we want to checks
scr = screens[screens.lng.isnull()==False][['lat','lng']]

In [237]:
screen_list = list(zip(scr['lat'],scr['lng']))

In [263]:
find_closest((38.876059,-77.020240), screen_list)

8.65658792939013e-06

In [261]:
screens[screens['Screen Addresss (City)']=='Silver Spring']

,Screen Design: Screen Design,Opportunity: Opportunity Name,Screen Placement Address,Screen Addresss (City),Screen Address (State),Screen Address (Zip),Install Stage,Region,Screen Type,lat,lng,Address,close_date
193,Discovery HQ Silver Spring Screen,Discovery HQ - Silver Spring PILOT,1 Discovery Place,Silver Spring,MD,20910,Customer Live,DC,Public Screen,38.995472,-77.028290,1 Discovery Place Silver Spring MD 20910,NaN
300,Central Silver Spring - Grosvenor,Central Silver Spring - Grosvenor,8455 Fenton St,Silver Spring,MD,20910,Customer Live,MD,Public Screen,38.995071,-77.023972,8455 Fenton St Silver Spring MD 20910,9/25/17
307,Silver Spring,MOCO 4 Regional SVC,One Veterans Place,Silver Spring,MD,20910,Customer Live,MD,Public Screen,38.997418,-77.024508,One Veterans Place Silver Spring MD 20910,NaN
308,Engrain - The Fenwick - Silver Spring MD - 861...,The Fenwick (8616 2nd Ave Silver Spring),8616 2nd Ave,Silver Spring,MD,20910,Customer Live,MD,Public Screen,38.997318,-77.034363,8616 2nd Ave Silver Spring MD 20910,NaN


In [302]:
find_within_distance((33.782524200000005, -84.380598800000001), screen_list, radius=1000000)

(33.782524200000005, -84.3805988)
found self (33.782524200000005, -84.380598800000001)


542

In [309]:
model_data['lat_lng'] = list(zip(model_data['lat'],model_data['lng']))

In [313]:
model_data['closest_ts']=np.nan

In [314]:
for i, row in model_data.iterrows():
    closest_ts = find_closest(row.lat_lng, screen_list)
    model_data.loc[model_data['id']==row.id, 'closest_ts'] = closest_ts

In [321]:
model_data.sort_values('closest_ts',ascending=False)

,id,name,close_date,stage,address,lat,lng,ts_response,mobility_score,carshare,bikeshare,ridehailing,masstransit,lat_lng,closest_ts
1004,0061a00000EV5MB,LinkedIn Singapore Office - Singapore,11/1/16,Lost,Singapore 018983,1.279424,103.854182,{'data': {'mobilityScore': {'latitude': 1.2794...,0.0,0.000000,0.000000,0.000000,0.000000,"(1.2794239, 103.854182)",1.081300e+04
1619,0061a00000G59Mn,Clear Channel Singapore,10/1/16,Lost,"2 Temasek Blvd, Singapore 038982",1.293430,103.858809,{'data': {'mobilityScore': {'latitude': 1.2934...,0.0,0.000000,0.000000,0.000000,0.000000,"(1.29343, 103.8588092)",1.081195e+04
748,0061a00000HyoAr,Brookfield - TBD,5/31/17,Lost,"Level 23, 135 King Street, Sydney Australia Ne...",-33.869213,151.208818,{'data': {'mobilityScore': {'latitude': -33.86...,0.0,0.000000,0.000000,0.000000,0.000000,"(-33.8692127, 151.2088179)",8.166114e+03
123,0061a00000MhvNV,Cushman & Wakefield-271 17th Street,3/30/18,Contract Sent,"Chicago, IL",13.193887,-59.543198,{'data': {'mobilityScore': {'latitude': 13.193...,0.0,0.000000,0.000000,0.000000,0.000000,"(13.193887, -59.543198)",2.574507e+03
65,0061a00000MiXV4,ADP- Spain,3/31/18,Qualified Lead,"Camí Antic de València, 54, 08005 Barcelona, S...",41.404790,2.201678,{'data': {'mobilityScore': {'latitude': 41.404...,39.0,7.680000,0.000000,0.000000,0.000000,"(41.4047905, 2.2016779)",1.137791e+03
1531,0061a00000Fs3J7,Urban Investment Partners- Portfolio,7/25/16,Lost,"140 Q St NE #140b, Washington, DC 20002",40.871953,-98.016174,{'data': {'mobilityScore': {'latitude': 40.871...,0.0,0.000000,0.000000,0.000000,0.000000,"(40.8719526, -98.0161744)",5.960625e+02
967,0061a00000MiJGe,UIP - DC - Frequency,3/31/18,Contract Sent,"140 Q St NE #140b, Washington, DC 20002",40.871953,-98.016174,{'data': {'mobilityScore': {'latitude': 40.871...,0.0,0.000000,0.000000,0.000000,0.000000,"(40.8719526, -98.0161744)",5.960625e+02
961,0061a00000KkNd5,STACK Real Estate- Salt Lake CIty,11/15/17,Lost,"2801 North Thanksgiving Way #100, Lehi, UT 84043",40.422339,-111.883408,{'data': {'mobilityScore': {'latitude': 40.422...,45.0,0.000000,0.000000,7.948718,3.800595,"(40.4223388, -111.8834079)",5.831657e+02
731,0061a00000HwQJW,Elsey Partners - 117 27th Ave Apartments - MSP,5/31/17,Lost,1532 College Ave. F19 Manhattan United States ...,39.196354,-96.597406,{'data': {'mobilityScore': {'latitude': 39.196...,0.0,0.000000,0.000000,0.000000,0.000000,"(39.1963544, -96.597406)",5.566318e+02
1020,0061a00000ItSRl,Titan Development- Albuquerque,7/31/17,Lost,"6300 Riverside Plaza Ln #200, Albuquerque, NM ...",35.156086,-106.679868,{'data': {'mobilityScore': {'latitude': 35.156...,49.0,0.000000,0.000000,9.230769,5.143068,"(35.1560861, -106.6798676)",5.153990e+02


In [324]:
model_data['within_one_tenth']=np.nan
model_data['within_one_half']=np.nan
model_data['within_one']=np.nan
model_data['within_five']=np.nan

In [328]:
for i, row in model_data.iterrows():
    within_one_tenth = find_within_distance(row.lat_lng, screen_list, .1)
    within_one_half = find_within_distance(row.lat_lng, screen_list, .5)
    within_one = find_within_distance(row.lat_lng, screen_list, 1)
    within_five = find_within_distance(row.lat_lng, screen_list, 5)
    model_data.loc[model_data['id']==row.id, 'within_one_tenth'] = within_one_tenth
    model_data.loc[model_data['id']==row.id, 'within_one_half'] = within_one_half
    model_data.loc[model_data['id']==row.id, 'within_one'] = within_one
    model_data.loc[model_data['id']==row.id, 'within_five'] = within_five
    

In [331]:
model_data.head(100)

,id,name,close_date,stage,address,lat,lng,ts_response,mobility_score,carshare,bikeshare,ridehailing,masstransit,lat_lng,closest_ts,within_one_tenth,within_one_half,within_one,within_five
0,0061a00000GcQdL,525 Water St SW- CMC Mgmt,3/8/18,Contract Sent,525 Water St SW Washington United States Distr...,38.876059,-77.020240,{'data': {'mobilityScore': {'latitude': 38.876...,86.0,12.190237,21.710327,21.000000,25.347332,"(38.876059, -77.0202399)",7.073231e-13,1.0,6.0,11.0,72.0
1,0061a00000Mi8dC,Bozzuto - NJ - Park + Garden,6/30/18,Invoice Sent,1450 Garden St Hoboken United States New Jerse...,40.754448,-74.028046,{'data': {'mobilityScore': {'latitude': 40.754...,85.0,3.435388,25.069638,15.000000,33.883246,"(40.7544482, -74.028046)",1.927253e-01,0.0,1.0,3.0,31.0
2,0061a00000M4PAU,Greystar- The Cameron,11/17/17,Deal Signed,"8710 Cameron St, Silver Spring, MD 20910",38.999289,-77.028885,{'data': {'mobilityScore': {'latitude': 38.999...,91.0,2.921386,8.266361,12.115385,70.074879,"(38.9992886, -77.0288854)",2.301765e-01,0.0,4.0,6.0,11.0
3,0061a00000MgelJ,JLL- 2035 NW Front Ave,1/31/18,Proposal Sent,"1120 NW Couch St #500, Portland, OR 97209",45.537557,-122.688094,{'data': {'mobilityScore': {'latitude': 45.537...,100.0,0.000000,0.000000,21.000000,1.468259,"(45.5375567, -122.6880937)",6.848705e-01,0.0,0.0,1.0,6.0
4,0061a00000KOF8z,International Monetary Fund- Spring Conference,1/31/18,Closed Won,"1900 Pennsylvania Ave NW, Washington, DC 20431",38.898940,-77.044101,{'data': {'mobilityScore': {'latitude': 38.898...,100.0,14.075039,40.000111,21.000000,87.423132,"(38.8989396, -77.0441015)",2.050043e-01,0.0,4.0,11.0,92.0
5,0061a00000Mgpep,GAP HQ- 1 Harrison St,2/2/18,Deal Signed,"1 Harrison St, San Francisco, CA 94105",37.790608,-122.390911,{'data': {'mobilityScore': {'latitude': 37.790...,100.0,5.058178,21.889612,15.000000,237.659368,"(37.790608, -122.3909111)",1.694180e-01,0.0,1.0,3.0,20.0
6,0061a00000HIaGb,AVB - AVA Theater - Boston,4/1/17,Invoice Sent,45 Stuart Street Boston United States Massachu...,42.351477,-71.063937,{'data': {'mobilityScore': {'latitude': 42.351...,100.0,6.295082,13.407821,15.000000,116.615340,"(42.3514766, -71.0639373)",5.972617e-01,0.0,0.0,3.0,38.0
7,0061a00000Mgdqw,Metro Village- Spring Place- DC,1/30/18,Proposal Sent,"3 Bethesda Metro, Suite 110 Bethesda United St...",38.984468,-77.095870,{'data': {'mobilityScore': {'latitude': 38.984...,85.0,11.802579,10.428737,15.000000,39.238321,"(38.9844678, -77.0958696)",1.249591e-01,0.0,4.0,5.0,8.0
8,0061a00000M4P5Q,Greystar- Aventine at Fort Totten,11/17/17,Deal Signed,"5210 3rd St NE, Washington, DC 20011",38.952866,-77.001508,{'data': {'mobilityScore': {'latitude': 38.952...,95.0,15.689538,16.853933,21.000000,54.083021,"(38.9528664, -77.0015081)",2.549615e-01,0.0,1.0,2.0,13.0
9,0061a00000M3j9z,Wood Partners- Alta Roosevelt- CHI,10/27/17,Closed Won,"801 S Financial Pl, Chicago, IL 60605",41.871918,-87.632974,{'data': {'mobilityScore': {'latitude': 41.871...,94.0,4.418025,16.981132,15.000000,67.967189,"(41.8719175, -87.6329737)",2.628677e-02,1.0,1.0,3.0,13.0


In [332]:
df.to_csv('model_data.csv', index=False)